In [15]:
import random

import numpy as np
import tensorflow as tf
from sklearn import metrics
from tensorflow.keras import layers, models

import matplotlib.pyplot as plt

ACTIVATION = 'relu'
SEED_VALUE = 42
NUM_CLASSES = 10

random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
tf.random.set_seed(SEED_VALUE)

In [16]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [17]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [18]:
def build_model():
    return models.Sequential([
        layers.Flatten(input_shape=(28, 28)), 
        layers.Dense(128, activation=ACTIVATION), 
        layers.Dense(64, activation=ACTIVATION),   
        layers.Dense(NUM_CLASSES)                      
    ])

In [19]:
def plot_history(history):
    fig, ax = plt.subplots(1, 5, figsize=(15, 5))
    fig.tight_layout(pad=5.0)

    ax[0].plot(history.history['accuracy'], label='accuracy')
    ax[0].plot(history.history['val_accuracy'], label='val_accuracy')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('Accuracy')
    ax[0].set_ylim([0, 1])

    ax[1].plot(history.history['precision'], label='precision')
    ax[1].plot(history.history['val_precision'], label='val_precision')
    ax[1].set_xlabel('Epoch')
    ax[1].set_ylabel('Precision')
    ax[1].set_ylim([0, 1])

    ax[2].plot(history.history['recall'], label='recall')
    ax[2].plot(history.history['val_recall'], label='recall')
    ax[2].set_xlabel('Epoch')
    ax[2].set_ylabel('Recall')
    ax[2].set_ylim([0, 1])

    history.history['f1_score'] = (2 * np.array(history.history['precision']) * np.array(history.history['recall']) /
                                    (np.array(history.history['precision']) + np.array(history.history['recall']))).tolist()

    history.history['val_f1_score'] = (2 * np.array(history.history['val_precision']) * np.array(history.history['val_recall']) /
                                    (np.array(history.history['val_precision']) + np.array(history.history['val_recall']))).tolist()

    ax[3].plot(history.history['f1_score'], label='f1_score')
    ax[3].plot(history.history['val_f1_score'], label='val_f1_score')
    ax[3].set_xlabel('Epoch')
    ax[3].set_ylabel('F1Score')
    ax[3].set_ylim([0, 1])

    ax[4].plot(history.history['loss'], label='accuracy')
    ax[4].plot(history.history['val_loss'], label='val_accuracy')
    ax[4].set_xlabel('Epoch')
    ax[4].set_ylabel('Loss')
    ax[4].set_ylim([0, 3])

    plt.show()

In [20]:
def custom_prec_score(y_true, y_pred):
    y_true=y_true.numpy()
    y_pred=y_pred.numpy()
    y_pred=np.argmax(y_pred, axis=-1)
    return precision_score(y_true, y_pred,average='macro')

def recall(y_true, y_pred):
    true_positives = tf.reduce_sum(tf.cast(tf.math.logical_and(tf.equal(y_true, 1), tf.equal(y_pred, 1)), tf.float32))
    actual_positives = tf.reduce_sum(tf.cast(tf.equal(y_true, 1), tf.float32))
    return true_positives / (actual_positives + tf.keras.backend.epsilon())

def f1_score(y_true, y_pred):
    precision_value = precision(y_true, y_pred)
    recall_value = recall(y_true, y_pred)
    return 2 * ((precision_value * recall_value) / (precision_value + recall_value + tf.keras.backend.epsilon()))

In [21]:
def test_model(model, optimizer):
    model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    print(f'\nTesting: {optimizer}')
    history = model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels), verbose=False)
    
    reached_90 = next((i + 1 for i, e in enumerate(history.history["val_accuracy"]) if e >= 0.9), np.inf) 

    if reached_90 == np.inf:
        print('\nNever reached 90% accuracy')
    else:
        print(f'\nReached 90% accuracy in {reached_90} epochs')

    
    results = model.predict(test_images)
    
    print(metrics.classification_report(np.argmax(results, axis=-1), test_labels))

    return (history, model)

In [22]:
optimizers = ['SGD', 'Adam', 'RMSprop', 'Adagrad']
nn_models = {}
training_results = {}

for optimizer in optimizers:
    model = build_model()
    nn_models[optimizer] = model

    training_results[optimizer] = test_model(model, optimizer)


Testing: SGD

Never reached 90% accuracy
313/313 [==============================] - 0s 866us/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.11      0.20     10000
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

    accuracy                           0.11     10000
   macro avg       0.10      0.01      0.02     10000
weighted avg       1.00      0.11      0.20     10000

Testing: Adam


/home/volodymyr/.local/share/virtualenvs/python_neural_nets-VlmVW8YE/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/volodymyr/.local/share/virtualenvs/python_neural_nets-VlmVW8YE/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/volodymyr/.local/share/virtualenvs/python_neural_nets-VlmVW8YE/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` param


Reached 90% accuracy in 1 epochs
313/313 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       977
           1       0.99      0.99      0.99      1134
           2       0.97      0.97      0.97      1036
           3       0.98      0.95      0.97      1035
           4       0.97      0.98      0.97       973
           5       0.97      0.95      0.96       903
           6       0.96      0.99      0.98       933
           7       0.96      0.98      0.97      1003
           8       0.96      0.95      0.96       981
           9       0.97      0.96      0.97      1025

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000


Testing: RMSprop

Reached 90% accuracy in 1 epochs
313/313 [==============================] - 0s 922us/step
              precision    recall  f1-score

In our investigation we`ve compared 4 optimizers - Adam, SGD, RMSprop and Adagard.
Ranking from the worst to the best optimizer based on accuracy, we have 
1) Stochastic Gradient Descend, which shows minimal performance, having 0.11 accuracy.
    SGD is considered one of the simplest optimizers, which may be good for really large data sets, as it uses only subset of validation data for weight-tuning. We also did not consider learning rate parameter, which value may be crucial to SGD`s performance
2) Adagard - 